# Imports

In [98]:
import os
from pyMez import *
from pyMez.Code.Analysis.SParameter import *
from pyMez.Code.DataHandlers.NISTModels import *
from pyMez.Code.DataHandlers.Translations import *
import sys
sys.path.insert(0, '..')
#from Machine_Learning import *
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import pickle

# Setting up the Dictionary System and opening the files needed to train the classifer

In [4]:
device_type_dictionary={0:"3.3KOhms",1:"3.6KOhms",2:"3.9KOhms",3:"100KOhms",4:"200KOhms",
                        5:"ZenerDiode",6:"GreenLED"}
pretrain_folder=r"C:\Users\msh7\Desktop\KiethleyControl"
pretrain_files=os.listdir(pretrain_folder)
pretrain_files=map(lambda x:os.path.join(pretrain_folder,x),pretrain_files) 








# Function that creates the training arrays (increases the number of training sets)

In [115]:
def create_xml_training_set(pretrain_folder,number_training_sets=1000,expansion_factor=1):
    """Given a directory containing multiple measurements this creates a list of np.arrays to 
    train a classifier"""
    pretrain_files=os.listdir(pretrain_folder)
    pretrain_files=map(lambda x:os.path.join(pretrain_folder,x),pretrain_files)
    # now open the files convert them to AsciiData Tables and find the mean and standard deviation
    first_file=XmlDataTable_to_AsciiDataTable((DataTable(pretrain_files[0])))
    joined_file=first_file.copy()
    for file_name in pretrain_files[1:]:
        joined_file=joined_file+XmlDataTable_to_AsciiDataTable(DataTable(file_name))
    #replaces the column Index with Frequency 
    joined_file.column_names=map(lambda x: x.replace("Index","Frequency"),joined_file.column_names)
    #Turns the columns into floats 
    joined_file.options["column_types"] = ["float" for i in joined_file.column_names]
    joined_file.data=[[float(joined_file.data[j][i]) for i in range(len(row))] for j,row in enumerate(joined_file.data[:])]
    mean_file=frequency_model_collapse_multiple_measurements(joined_file)
    std_file=frequency_model_collapse_multiple_measurements(joined_file,
                                                            method="std")
    #calucates the mean and standard deviation 
    mean_array=np.array(mean_file["Current"])
    std_array=np.array(std_file["Current"])
    #increases the number of training arrays 
    training_arrays=np.array([np.random.normal(loc=mean_value,scale=expansion_factor*std_array[index]+.0001,
                                               size=number_training_sets) for index,
                              mean_value in enumerate(mean_array)])
    #reorganizes the training arrays 
    training_arrays=[[training_arrays[i][j] 
                      for i in range(len(mean_array))] for j in range(number_training_sets)]
    return training_arrays

# Trains each individual device

In [116]:
training_set33=create_xml_training_set(r"C:\Users\msh7\Desktop\KiethleyControl\resistor_3.3kohm")
training_set36=create_xml_training_set(r"C:\Users\msh7\Desktop\KiethleyControl\resistor_3.6kohm")
training_set39=create_xml_training_set(r"C:\Users\msh7\Desktop\KiethleyControl\resistor_3.9kohm")
training_set100=create_xml_training_set(r"C:\Users\msh7\Desktop\KiethleyControl\resistor_100Kohm")
training_set200=create_xml_training_set(r"C:\Users\msh7\Desktop\KiethleyControl\resistor_200Kohm")
training_set_zener=create_xml_training_set(r"C:\Users\msh7\Desktop\KiethleyControl\diode_zener")
training_set_green=create_xml_training_set(r"C:\Users\msh7\Desktop\KiethleyControl\diode_greenLED")

# Adds all the individual training sets into one single array to train the classifer

In [117]:
training_array = []
training_array = training_set33+ training_set36 +training_set39+training_set100+training_set200+training_set_zener+ training_set_green

# Plots the training set of an individual device

In [ ]:
    plt.plot(training_set)
plt.show


for training_set in training_set36:
    plt.plot(training_set)
plt.show()

# Creates the target array that matches each device measurement with the corresponding number from the dictionary. Ex. 3.3KOhms measurements are all 0s

In [118]:
number_training_sets=1000
expansion_factor=1
sorted_device_keys=sorted(device_type_dictionary.keys())
target_array=[]
for key_index,key in enumerate(sorted_device_keys):
    target_array=target_array+[key for i in range(number_training_sets)]


# Trains the classifier

In [119]:
classifier=QuadraticDiscriminantAnalysis()
classifier.fit(training_array,target_array)
classifier.predict(training_array)


array([0, 0, 0, ..., 6, 6, 6])

In [120]:
len(training_array[0])

160

# Tests the classifier

In [121]:
xml = DataTable(r"C:\Users\msh7\Desktop\Hola\1234.xml")
test_file = XmlDataTable_to_AsciiDataTable(xml)
test_array = [np.array(test_file["Current"], dtype = float)]

predictions=classifier.predict(test_array)
devices=[device_type_dictionary[key] for key in predictions.tolist()]
print(devices)

['100KOhms']


In [66]:
predictions

array([3])

# PICKLE

In [109]:
pickle_out = open("MachineLearning.pickle", "wb")
pickle.dump(classifier, pickle_out)
pickle_out.close()

In [110]:
pickle_in = open("MachineLearning.pickle", "rb")

In [111]:
classifier = pickle.load(pickle_in)

In [112]:
predictions=classifier.predict(test_array)

In [113]:
devices=[device_type_dictionary[key] for key in predictions.tolist()]
print(devices)

['ZenerDiode']
